In [ ]:
#KP Genetico
import random

# Establecer los índices de la matriz de datos
# 0 = nombre, 1 = peso, 2 = valor, 3 = fitness
KNAPSACK_ITEM_NAME_INDEX = 0
KNAPSACK_ITEM_WEIGHT_INDEX = 1
KNAPSACK_ITEM_VALUE_INDEX = 2

# Pequeño conjunto de datos
#knapsack_items = [
#     ['Pearls', 3, 4],
#     ['Gold', 7, 7],
#     ['Crown', 4, 5],
#     ['Coin', 1, 1],
#     ['Axe', 5, 4],
#     ['Sword', 4, 3],
#     ['Ring', 2, 5],
#     ['Cup', 3, 1],]

# Gran conjunto de datos
# knapsack_items = [peso, valor]
knapsack_items = [
    ['Axe', 32252, 68674],
    ['Bronze coin', 225790, 471010],
    ['Crown', 468164, 944620],
    ['Diamond statue', 489494, 962094],
    ['Emerald belt', 35384, 78344],
    ['Fossil', 265590, 579152],
    ['Gold coin', 497911, 902698],
    ['Helmet', 800493, 1686515],
    ['Ink', 823576, 1688691],
    ['Jewel box', 552202, 1056157],
    ['Knife', 323618, 677562],
    ['Long sword', 382846, 833132],
    ['Mask', 44676, 99192],
    ['Necklace', 169738, 376418],
    ['Opal badge', 610876, 1253986],
    ['Pearls', 854190, 1853562],
    ['Quiver', 671123, 1320297],
    ['Ruby ring', 698180, 1301637],
    ['Silver bracelet', 446517, 859835],
    ['Timepiece', 909620, 1677534],
    ['Uniform', 904818, 1910501],
    ['Venom potion', 730061, 1528646],
    ['Wool scarf', 931932, 1827477],
    ['Cross bow', 952360, 2068204],
    ['Yesteryear book', 926023, 1746556],
    ['Zinc cup', 978724, 2100851, 0]
]

# La mejor puntuación de la mochila a partir del enfoque de fuerza bruta
BEST_LARGE_KNAPSACK_SCORE = 13692887

# Los algoritmos genéticos se utilizan para evaluar grandes espacios de búsqueda de una buena solución. Es importante tener en cuenta que un
# algoritmo genético no está garantizado para encontrar la mejor solución absoluta. Intenta encontrar la mejor solución global mientras
# evitar las mejores soluciones locales. El ciclo de vida general de un algoritmo genético es el siguiente:

# - Creación de una población: Se trata de crear una población aleatoria de soluciones potenciales.

#  - Medición de la aptitud de los individuos de la población: Se trata de determinar la calidad de una solución específica.
# Esto se logra mediante el uso de una función de aptitud que puntúa las soluciones para determinar lo buenas que son.

# - Seleccionar a los padres en función de su aptitud: Se trata de seleccionar un número de parejas de progenitores que reproduzcan
# descendencia.

# - Reproducción de individuos a partir de los padres: Se trata de crear una descendencia a partir de sus respectivos progenitores mezclando
# información genética y aplicando ligeras mutaciones a la descendencia.

# - Poblar la siguiente generación: Esto implica la selección de individuos y descendientes de la población que
# sobrevivan a la siguiente generación.

# Los índices de las propiedades de un individuo
INDIVIDUAL_CHROMOSOME_INDEX = 0
INDIVIDUAL_FITNESS_INDEX = 1
INDIVIDUAL_PROBABILITY_INDEX = 2


# Generar una población inicial de individuos aleatorios
def generate_initial_population(population_size):
    population = []
    for individual in range(0, population_size):
        individual = ''.join([random.choice('01') for n in range(26)])
        population.append([individual, 0, 0])
    return population


# Calcular la aptitud de cada individuo de la población dado el peso máximo
def calculate_population_fitness(population, maximum_weight):
    best_fitness = 0
    for individual in population:
        individual_fitness = calculate_individual_fitness(individual[INDIVIDUAL_CHROMOSOME_INDEX], maximum_weight)
        individual[INDIVIDUAL_FITNESS_INDEX] = individual_fitness
        if individual_fitness > best_fitness:
            best_fitness = individual_fitness
        if individual_fitness == -1:
            population.remove(individual)
    return best_fitness


# Calcular la aptitud de un individuo
def calculate_individual_fitness(individual, maximum_weight):
    total_individual_weight = 0
    total_individual_value = 0
    for gene_index in range(len(individual)):
        gene_switch = individual[gene_index]
        if gene_switch == '1':
            total_individual_weight += knapsack_items[gene_index][KNAPSACK_ITEM_WEIGHT_INDEX]
            total_individual_value += knapsack_items[gene_index][KNAPSACK_ITEM_VALUE_INDEX]
    if total_individual_weight > maximum_weight:
        return -1
    return total_individual_value


# Establecer las probabilidades de selección para cada individuo de la población
def set_probabilities(population):
    population_sum = sum(individual[INDIVIDUAL_FITNESS_INDEX] for individual in population)
    for individual in population:
        individual[INDIVIDUAL_PROBABILITY_INDEX] = individual[INDIVIDUAL_FITNESS_INDEX] / population_sum


# Selección por ruleta para seleccionar individuos en una población
def roulette_wheel_selection(population, number_of_selections):
    set_probabilities(population)
    slices = []
    total = 0
    for r in range(0, len(population)):
        individual = population[r]
        slices.append([r, total, total + individual[INDIVIDUAL_PROBABILITY_INDEX]])
        total += individual[INDIVIDUAL_PROBABILITY_INDEX]
    chosen_ones = []
    for r in range(number_of_selections):
        spin = random.random()
        result = [s[0] for s in slices if s[1] < spin <= s[2]]
        chosen_ones.append(population[result[0]])
    return chosen_ones


# Reproducir hijos dados dos individuos utilizando un cruce de puntos
def one_point_crossover(parent_a, parent_b, xover_point):
    children = [parent_a[:xover_point] + parent_b[xover_point:],
                parent_b[:xover_point] + parent_a[xover_point:]]
    return children


# Reproducir hijos a partir de dos individuos mediante un cruce de dos puntos
def two_point_crossover(parent_a, parent_b, xover_point_1, xover_point_2):
    children = [parent_a[:xover_point_1] + parent_b[xover_point_1:xover_point_2] + parent_a[xover_point_2:],
                parent_b[:xover_point_1] + parent_a[xover_point_1:xover_point_2] + parent_b[xover_point_2:]]
    return children


# Mutaciones aleatorias en los hijos
def mutate_children(children, mutation_rate):
    for child in children:
        random_index = random.randint(0, mutation_rate)
        if child[INDIVIDUAL_CHROMOSOME_INDEX][random_index] == '1':
            mutated_child = list(child[INDIVIDUAL_CHROMOSOME_INDEX])
            mutated_child[random_index] = '0'
            child[INDIVIDUAL_CHROMOSOME_INDEX] = mutated_child
        else:
            mutated_child = list(child[INDIVIDUAL_CHROMOSOME_INDEX])
            mutated_child[random_index] = '1'
            child[INDIVIDUAL_CHROMOSOME_INDEX] = mutated_child
    return children


# Reproducir a los hijos dados los individuos seleccionados
def reproduce_children(chosen_selections):
    children = []
    for parent_index in range(len(chosen_selections)//2 - 1):
        children = one_point_crossover(chosen_selections[parent_index],
                                       chosen_selections[parent_index + 1],
                                       CROSSOVER_POSITION_1)
    return children


# Combinar la población existente y los hijos recién reproducidos
def merge_population_and_children(population, children):
    return population + children


# Establecer los hiperparámetros para el algoritmo genético
NUMBER_OF_GENERATIONS = 1000
INITIAL_POPULATION_SIZE = 1000
KNAPSACK_WEIGHT_CAPACITY = 6404180
CROSSOVER_POSITION_1 = 13
CROSSOVER_POSITION_2 = 22
MUTATION_RATE = 10
NUMBER_OF_ITERATIONS = 5


# Ejecutar el algoritmo genético
def run_ga():
    best_global_fitness = 0
    global_population = generate_initial_population(INITIAL_POPULATION_SIZE)
    for generation in range(NUMBER_OF_GENERATIONS):
        current_best_fitness = calculate_population_fitness(global_population, KNAPSACK_WEIGHT_CAPACITY)
        if current_best_fitness > best_global_fitness:
            best_global_fitness = current_best_fitness
        the_chosen = roulette_wheel_selection(global_population, 100)
        the_children = reproduce_children(the_chosen)
        the_children = mutate_children(the_children, MUTATION_RATE)
        global_population = merge_population_and_children(global_population, the_children)


    print('Mejor fitness: ', best_global_fitness)
    print('Mejor Actual: ', BEST_LARGE_KNAPSACK_SCORE)
    print('Exactitud: ', best_global_fitness / BEST_LARGE_KNAPSACK_SCORE * 100)
    print('Tamaño final de la población: ', len(global_population))
    print(global_population)



# Ejecutar el algoritmo genético durante un número de iteraciones
for i in range(0, NUMBER_OF_ITERATIONS):
    run_ga()


#print(calculate_individual_fitness('01100100010110001110001001', 6404180))
# print(calculate_individual_fitness('00110101000100011010001000', 6404180))
# print(calculate_individual_fitness('11100100110110000100101101', 6404180))
# print(calculate_individual_fitness('00001000010010101101001001', 6404180))

Mejor fitness:  13454767
Mejor Actual:  13692887
Exactitud:  98.26099492386084
Tamaño final de la población:  2007
[['10101010100111110010001100', 12854833, 0.0005853806262493952], ['00000110000001111011010000', 10352128, 0.0004714129830900097], ['10010000010010001011001010', 9617816, 0.00043797404083207095], ['00110001010010000100011010', 11152894, 0.0005078780933375892], ['10000001110000001101100000', 10710006, 0.0004877099546462237], ['00100001111100011011000000', 12597905, 0.0005736806941272894], ['10111101000101001111100000', 12598753, 0.0005737193101692917], ['00110001011001101011000100', 12883222, 0.0005866733984385473], ['10111000101001000010110000', 9095385, 0.0004141836900782262], ['11010100000110000010110100', 9380440, 0.00042716446348971444], ['10001011011011011010110000', 12418401, 0.0005655064795004427], ['11000000010010100000101001', 8787848, 0.00040017913617582546], ['11011111000011100010100110', 13063179, 0.000594868241682171], ['00010010110001001000010000', 7835001, 0

**Algoritmo Genetico**


[ENLACE](https://drive.google.com/uc?id=1E13oRIQaT7NZJz724Jeh05vW2C4LI8-Q)